In [2]:
import openpyxl
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os

# Initialize ChromeDriver
service = Service('D:/Assignment/chromedriver.exe')  # Make sure this path is correct
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    driver = webdriver.Chrome(service=service, options=options)
    print("ChromeDriver initialized successfully.")
except Exception as e:
    print(f"Error initializing WebDriver: {e}")
    exit()

# Load the workbook and select the sheet based on the current day
workbook_path = 'D:/Assignment/4BeatsQ1.xlsx'
try:
    workbook = openpyxl.load_workbook(workbook_path)
except Exception as e:
    print(f"Error opening Excel file: {e}")
    driver.quit()
    exit()

current_day = datetime.now().strftime('%A')  # e.g., 'Friday'
if current_day in workbook.sheetnames:
    sheet = workbook[current_day]
else:
    print(f"Error: Worksheet '{current_day}' does not exist in the Excel file.")
    driver.quit()
    exit()

# Function to perform Google search and return longest and shortest suggestions
def search_google(driver, keyword):
    try:
        driver.get("https://www.google.com/")
        print(f"Searching for: {keyword}")

        # Accept cookies if prompted
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="L2AGLb"]'))).click()
        except TimeoutException:
            print("No cookie prompt found or it took too long to appear.")
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.clear()
        search_box.send_keys(keyword)

        # Wait for suggestions to load
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ul[role="listbox"] li span')))
        suggestions = driver.find_elements(By.CSS_SELECTOR, 'ul[role="listbox"] li span')
        options = [suggestion.text for suggestion in suggestions if suggestion.text]
        if options:
            longest_option = max(options, key=len)
            shortest_option = min(options, key=len)
            return longest_option, shortest_option
    except Exception as e:
        print(f"Error searching for keyword '{keyword}': {e}")
    return None, None

# Read keywords from column C of the sheet
keywords = [row[0] for row in sheet.iter_rows(min_row=2, min_col=3, max_col=3, values_only=True) if row[0]]

# Dictionary to store results
results = {}
# Call the search_google() for each keyword and store the result
for keyword in keywords:
    longest, shortest = search_google(driver, keyword)
    results[keyword] = {'longest': longest, 'shortest': shortest}
    print(f"Keyword: {keyword}, Longest: {longest}, Shortest: {shortest}")

# Close the browser
driver.quit()

# Write results back to the Excel sheet in columns D (Longest Option) and E (Shortest Option)
for row in sheet.iter_rows(min_row=2, min_col=3, max_col=5):
    keyword = row[0].value
    if keyword in results:
        sheet.cell(row=row[0].row, column=4).value = results[keyword]['longest']
        sheet.cell(row=row[0].row, column=5).value = results[keyword]['shortest']

# Save the workbook
output_path = 'D:/Assignment/4BeatsQ1.xlsx'
try:
    workbook.save(output_path)
    print(f"Results saved to {output_path}")
except Exception as e:
    print(f"Error saving Excel file: {e}")


ChromeDriver initialized successfully.
Searching for: Dhaka
No cookie prompt found or it took too long to appear.
Keyword: Dhaka, Longest: মাধ্যমিক ও উচ্চমাধ্যমিক শিক্ষা বোর্ড, ঢাকা · 5 জয়নাগ রোড, ঢাকা, Shortest: ঢাকা
Searching for: Saturday
No cookie prompt found or it took too long to appear.
Keyword: Saturday, Longest: স্যাটারডে নাইট লাইভ — টিভি প্রোগ্রাম, Shortest: saturday
Searching for: Baby
No cookie prompt found or it took too long to appear.
Keyword: Baby, Longest: জাস্টিন বিবার এবং লুডাক্রিস এর গান, Shortest: গান
Searching for: School
No cookie prompt found or it took too long to appear.
Keyword: School, Longest: school college kobe khulbe, Shortest: school
Searching for: Cricket
No cookie prompt found or it took too long to appear.
Keyword: Cricket, Longest: আইসিসি পুরুষ টি২০ বিশ্বকাপ — চ্যাম্পিয়নশিপ, Shortest: cricket
Searching for: Momey
No cookie prompt found or it took too long to appear.
Keyword: Momey, Longest: money cannot bring happiness story, Shortest: money
Searc